In [1]:
import ginsim
import biolqm
from colomoto_jupyter import tabulate

import pandas as pd
import numpy as np

In [2]:
lrg = ginsim.load("IFN_covid.zginml")
m = ginsim.to_biolqm(lrg)

# The status of SARS_nsp3_SUD is not clear, ignore this input for now
m = biolqm.perturbation(m, "SARS_nsp3_SUD%0")

ginsim.show(lrg)

Then we define some helper functions to visualize fixed components (derived from the analysis of T cell inhibitory checkpoints by Celine Hernandez)

In [4]:
# Transforms a dictionary into a dash-like pattern used for space restrictions.
# If a model has 4 components A, B, C, D in this order,
#  {A:0, D:1} => "0--1"
def dash_pattern(model, dict_vals):
    specific_comps = dict_vals.keys()
    str_pattern = ""
    for comp in model.getComponents():
        if comp.toString() in specific_comps:
            str_pattern += str(dict_vals.get(comp.toString()))
        else :
            str_pattern += "-"
    return(str_pattern)

def restrict_model(model, **dict_vals):
    pattern = dash_pattern(model, dict_vals)
    return biolqm.restrict(model, pattern)

def fill_fixed(data, names, functions, mddman):
    all_values = [f for f in functions]
    for comp, func in zip(names, functions):
        if mddman.isleaf(func): data[comp] = func
        else: data[comp] = -1
    

def get_fixed_pattern(all_names, model, as_dict=False):
    # Build a container for the results
    pattern = {key: 100 for key in all_names}
    
    # Model manager and core components
    mddman = model.getMDDManager()
    core_components = [node.getNodeID() for node in model.getComponents()]
    extra_components = [node.getNodeID() for node in model.getExtraComponents()]
    
    # 1/ Non-extra values: if the model was not reduced, core components may also contain fixed values
    fill_fixed(pattern, core_components, model.getLogicalFunctions(), mddman)

    # Special value for input components
    for node in model.getComponents():
        if node.isInput():
            pattern[node.getNodeID()] = -2

    # 2/ Extra values : only available after reduction/percolation
    # Functions of each component
    fill_fixed(pattern, extra_components, model.getExtraLogicalFunctions(), mddman)

    if as_dict: return pattern
    return pd.Series(pattern, dtype=np.byte).values.tobytes()

def compare_fixed_pattern(all_names, model1, model2, as_dict=False):
    pattern1 = get_fixed_pattern(all_names, model1, as_dict=True)
    pattern2 = get_fixed_pattern(all_names, model2, as_dict=True)
    
    pattern = {}
    for c in pattern1:
        v1 = pattern1[c]
        v2 = pattern2[c]
        
        if v1 == v2: pattern[c] = v1
        elif v1 < 0: pattern[c] = 10 + v2
        elif v2 < 0: pattern[c] = 20 + v1
        else: pattern[c] = 100

    if as_dict: return pattern
    return pd.Series(pattern, dtype=np.byte).values.tobytes()


def show_fixed_comparison(gs_model, restricted_model1, restricted_model2, styler, save=None):
    name_components = [ n.getId() for n in gs_model.getNodeOrder() ]
    pattern = compare_fixed_pattern(name_components, restricted_model1, restricted_model2)
    styler.setState( pattern )
    return ginsim.show(gs_model, style=styler, save=save)    


def show_fixed(gs_model, restricted_model, styler, save=None):
    name_components = [ n.getId() for n in gs_model.getNodeOrder() ]
    fixed_pattern = get_fixed_pattern(name_components, restricted_model)
    styler.setState(fixed_pattern)
    return ginsim.show(gs_model, style=styler, save=save)


def restricted_outputs(m_restricted, outputs):
    fixed_components = [ c.getNodeID() for c in m_restricted.getExtraComponents() ]
    
    output_values = {}
    for o in outputs:
        output_value = -1
        if o in fixed_components:
            output_index = fixed_components.index(o)
            output_value = m_restricted.getExtraLogicalFunctions()[output_index]
        output_values[o] = output_value
    return output_values

# Define color mapping rules

# Style for a single fixed pattern
styler_fixed = ginsim.lrg_style(lrg)
styler_fixed.mapState2Color(0, 200, 25, 25)
styler_fixed.mapState2Color(1, 100, 175, 100)
styler_fixed.mapState2Color(2, 100, 225, 100)
styler_fixed.mapState2Color(-1, 255, 255, 255)
styler_fixed.mapState2Color(-2, 175, 175, 175)


# Style for comparing two patterns
styler_comp = ginsim.lrg_style(lrg)
styler_comp.mapState2Color(-2, 175, 175, 175) # INPUT: gray
styler_comp.mapState2Color(0, 255, 255, 180)  # OFF  in both: light yellow
styler_comp.mapState2Color(1, 255, 180, 120)  # ON   in both: light orange
styler_comp.mapState2Color(2, 255, 180, 120)  # HIGH in both: light orange
styler_comp.mapState2Color(-1, 255, 255, 255) # FREE in both: white
styler_comp.mapState2Color(10, 200, 255, 200) # OFF in the first: light green
styler_comp.mapState2Color(11, 125, 200, 125) # ON  in the first: dark green
styler_comp.mapState2Color(20, 200, 200, 255) # OFF in the second: light green
styler_comp.mapState2Color(21, 125, 125, 200) # ON  in the second: dark green
styler_comp.mapState2Color(100, 255, 180, 180) # Other (different values?): red

Identify core and input components

In [5]:
cpts = [ c.getNodeID() for c in m.getComponents() ]
inputs = [ c.getNodeID() for c in m.getComponents() if c.isInput() ]


core = [ c for c in cpts if c not in inputs ]

# Identify viral inputs but ignore SARS_nsp3_SUD for now
viral_inputs = [ c for c in inputs if  ("SARS" in c or "MERS" in c or "nsp3" in c) and "nsp3_SUD" not in c]


outputs = ("Innate_immunity_phenotype",)

show_fixed(lrg, m, styler_fixed)

# Fixing all inputs

In this preliminary model, the core components can be fixed by propagation when assigning a fixed value to all inputs.

If we set all inputs to 0 (absence of viral proteins and of external stimulation), then the innate response can be activated.

However if we add viral proteins, then they block the innate response.

In [6]:
# By default: fix all cellular inputs at 0 and viral inputs at 1
default_inputs = { **{c:0 for c in inputs}, **{c:1 for c in viral_inputs}}


m_restricted = restrict_model(m, **default_inputs)
show_fixed(lrg, m_restricted, styler_fixed)

# Input sensitivity analysis

We saw that the viral proteins can block the innate response. We can check if the removal of one of them is sufficient to remove this blocking.

Performing a simple sensitivity analysis shows that when we remove a single viral protein, the response remains blocked, except for the removal of nsp3, which is enough to restore the response.

The addition of a single cellular input does not allow to restore the response.

In [7]:
%%time

sensitivity = {o:{} for o in outputs}
for i in viral_inputs:
    fixed_inputs = {**default_inputs, i:0}
    m_restricted = restrict_model(m, **fixed_inputs)
    results = restricted_outputs(m_restricted, outputs)
    for o in outputs:
        sensitivity[o][i] = results[o]

pd.DataFrame(sensitivity)

CPU times: user 285 ms, sys: 159 ms, total: 444 ms
Wall time: 803 ms


,Innate_immunity_phenotype
MERS_4a,0
MERS_4a_4b,0
MERS_5,0
MERS_PLPRo,0
MERS_PLPro,0
PLpro_nsp3,0
SARS_3b_8b_8ab,0
SARS_9b,0
SARS_E,0
SARS_M,0


In [8]:
%%time

sensitivity = {o:{} for o in outputs}
for i in inputs:
    if i in viral_inputs: continue
    fixed_inputs = {**default_inputs, i:1}
    m_restricted = restrict_model(m, **fixed_inputs)
    results = restricted_outputs(m_restricted, outputs)
    for o in outputs:
        sensitivity[o][i] = results[o]

pd.DataFrame(sensitivity)

CPU times: user 130 ms, sys: 73.2 ms, total: 203 ms
Wall time: 364 ms


,Innate_immunity_phenotype
IFNA1_Extracellular_Space,0
IFNAR_complex,0
IFNB1_Extracellular_Space,0
PAMP_simple_molecule,0
RELA,0
TBK1,0
